# 8기 과제 - 딥러닝 기반 상품 카테고리 자동 분류 서버

## 과제 개요
* 출제자 : 남상협 멘토 (justin@buzzni.com) / 버즈니 (http://buzzni.com) 대표
* 배경 : 카테고리 분류 엔진은 실제로 많은 서비스에서 사용되는 중요한 기계학습 기술이다. 본 과제의 주제는 버즈니 개발 인턴이자 마에스트로 6기 멘티가 아래와 나와 있는 기본 분류 모델을 기반으로 deep learning 기반의 feature 를 더해서 고도화된 분류 엔진을 만들어서 2016 한국 정보과학회 논문으로도 제출 했던 주제이다. 기계학습에 대한 학습과, 실용성 두가지 측면에서 모두 도움이 될 것으로 보인다.


## 과제 목표
* 입력 : 상품명, 상품 이미지
* 출력 : 카테고리
* 목표 : 가장 높은 정확도로 분류를 하는 분류 엔진을 개발



## 평가 항목 
* 성능평가 (100%)
 
## 제출 항목 
* 채점 서버에 자신이 분류한 class id 리스트를 파라미터로 넣어서 호출한다. 
* name - 자신의 이름을 넣는다. 실제 점수판에는 공개가 안됨, 추후 평가시에 일치하는 이름의 멘티 점수로 사용함. 요청한 평가 중에서 가장 높은 점수의 평가 점수로 업데이트됨.
* nickname - 점수판에 공개되는 이름, 자신의 이름으로 해도 되고, 닉네임으로 해도 됨. 구분을 위해서 사용하는 feature(text, textimage) 와 알고리즘 (svm, cnn) 등을 닉네임 뒤에 붙여준다. 
* pred_list - 분류한 카테고리 id 리스트를 , 로 묶은 데이터 
* 평가 점수가 반환된다. - precision, 높을 수록 좋다. 두가지 방법 각각 50%씩 점수 반영 
* mode - 'test' 로 호출하면 웹으로 순위가 공개되는 테스트 평가를 수행하고 결과 점수가 반환된다. 해당 결과 점수는 http://eval.buzzni.net:20002/score 에서 확인 가능함. 실제 성적 평가는 'eval' 로 평가용 데이터로 호출하면 된다. 이때는 점수가 반환되거나, 웹 점수 보드에도 나오지 않는다. 
* 너무 자주 평가를 요청하기 보다, 가급적 자체적으로 평가 해서, 괜찮게 나올때 요청하길 권장 
```python
import requests
name='test1'
nickname='test1_text_svm'
mode='test' #'eval' 을 실제 성적 평가용. 분류 점수 반환 안됨.
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,"mode":mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())         
         ```

## 성능 향상 포인트
* http://localhost:8000/notebooks/maestro8_deeplearning_product_classifier.ipynb 이 노트북에 있는 딥러닝 기반의 분류기로 분류할 경우에 더 높은 성능을 낼 수 있어서 유리함
* 아래의 방법들은 하나의 예이고, 아래에 나와 있지 않은 다양한 방법들도 가능함.
* 전처리 
 * 오픈된 형태소 분석기(예 - konlpy) 를 써서, 단어 띄어쓰기를 의미 단위로 띄어서 학습하기
 * bigram, unigram, trigram 등 단어 feature 를 더 다양하게 추가하기
* 딥러닝 
 * embedding weight 를 random 이 아닌 학습된 값을 사용하기 (https://radimrehurek.com/gensim/models/word2vec.html)
 * 이미지 feature 를 CNN으로 추출할때 더 성능이 좋은 모델 사용하기 (예제로 준 데이터는 mobilenet 으로 성능보다 속도 위주로 된 모델)
 * 다양한 파라미터(hyper parameter) 로 실험 해보기 
* 피쳐 조합  
 * 이미지 feature 와 text feature 를 합치는 부분 잘하기 


## 평가 점수 서버 
* 현재 평가 순위를 json 형태로 반환한다.
* 여러번 호출했을때는 가장 높은 점수로 업데이트 한다.
 * http://eval.buzzni.net:20002/score
* 실제 점수는 

In [228]:
from konlpy.tag import Kkma, Twitter
from konlpy.corpus import kolaw
from konlpy.utils import pprint
from nltk import collocations

In [229]:
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
# from keras import backend
# from keras.layers import Dense, Input, Lambda, LSTM, TimeDistributed
# from keras.layers.merge import concatenate
# from keras.layers.embeddings import Embedding
# from keras.models import Model



In [230]:
import string, pickle

In [233]:
with open("patched_list_v3", 'rb') as f:
    eng = pickle.load(f)

In [232]:
test_eng[:100]

[('720988653', 'spray gear office opirus ruth light metal metal'),
 ('1378489203',
  'expo exponet co., ltd. net co., ltd. coil coil tap tap cutting tool tool working tool'),
 ('1621140511',
  'oga organic mamnick mam peng pengko nose shawl incheon incheon store'),
 ('1549548735',
  'golf golf distance measuring range golf distance measuring telescope'),
 ('1642215271', 'electric power'),
 ('1594181530', 'hand tools such as phillips screwdriver driver etc.'),
 ('1002465039',
  'toi toit tron tron sil silva baban anan family family kindergarten'),
 ('1592587497', 'nunuri new new tech tech seated bbq'),
 ('181355585',
  'tongue tongue cleaner cleaner separate cleaner mouth odor odor removal oral cleanser traveling toothpaste toothbrush toothbrush tooth toothpaste toothpaste'),
 ('1212013675',
  'wire wire band band tong hose hose band hose fastening fastening pipe pipe band iron iron band'),
 ('1218797115',
  'trace 1 tastas japan sachihatachihata meteorite meteorite magic meteorite mete

In [234]:
y_name_id_dict

{'가구/인테리어': 11,
 '가전': 12,
 '건강': 7,
 '도서/문구': 8,
 '디지털': 10,
 '반려동물': 4,
 '뷰티': 0,
 '생필품/주방': 13,
 '스포츠/레저': 9,
 '식품': 6,
 '여행/e쿠폰': 15,
 '의류': 2,
 '자동차/공구': 1,
 '잡화': 14,
 '출산/육아': 16,
 '취미': 5,
 '컴퓨터': 3}

In [238]:
y_name_id_dict.items(), y_list

(dict_items([('자동차/공구', 1), ('디지털', 10), ('취미', 5), ('뷰티', 0), ('도서/문구', 8), ('잡화', 14), ('출산/육아', 16), ('건강', 7), ('식품', 6), ('반려동물', 4), ('가구/인테리어', 11), ('여행/e쿠폰', 15), ('스포츠/레저', 9), ('생필품/주방', 13), ('컴퓨터', 3), ('가전', 12), ('의류', 2)]),
 [11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11

In [240]:
with open("/home/yunsoo/mysite-projects/text_cnn_fork/train_eng1.kkk", 'w') as f:
    for pid, name in eng:
        for idx, data in enumerate(X_train):
            if pid == data[0]:
                for key, val in y_name_id_dict.items():
                    if val == y_train[idx]:
                        category = key
                        print(category)
                        f.write( name.replace(',', ' ') + "," +category + "\n")
f.close()

가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어
가구/인테리어


In [186]:
for idx, data in enumerate(x_text_list[:10]):
    print(idx, data)

0 ('1234621373', '#MD추천정품# 심플베드 엘레강스 고급형 접이식침대 더블사이즈115cm/수동/등받이6단으로 원하는각도조절')
1 ('1543513007', '(9731200) 소형제도판 (S) 661 A3')
2 ('1546650266', '(AGCRIP 웨빙 포켓조끼 빅사이즈 조끼 MK321_324 아웃')
3 ('1589443530', '(ATK아텍스 전동접이식침대(BE556) 전동침대 간병용침')
4 ('1520510486', '(MST플래그 메세지 캔들 마더 향초 양초캔들 캔들선물 아로마캔들 향초캔들')
5 ('1512822613', '(광일체어) 스카치A형 3인 등무 장의자')
6 ('1525985492', '(광일체어) 스파고아우드 2인 등유 로비용장의자')
7 ('1326388222', '(라움스튜디오)스크류 블랙봉 (일반-2.5cm) / 블랙 커튼봉/ 25mm / 일반봉 / 커튼봉 / 블랙')
8 ('1426607155', '(모슬리메모리즈) 산토리니 브리즈 (중) 모슬리메모리즈 아로마캔들 디퓨저 소이캔들 소이왁스')
9 ('1334473563', '(무료배송)양키캔들 악세사리 윅디퍼 심지가위 라이터')


In [187]:
[1,2,3].remove(1)

In [188]:
def extract_noun(datas):
    kkma = Kkma()
    word_list = []
    for data in datas:
        line = kkma.nouns(data[1])
        for word in line:
            try: 
                int(word)
                line.remove(word)
            except:
                if word in string.punctuation:
                    line.remove(word)
        line = " ".join(line)
        word_list.append(line)
    return word_list

In [189]:
type("1234")

str

In [190]:
def twitter(datas):
    kkma = Twitter()
    word_list = []
    for data in datas:
        line = kkma.nouns(data[1])
        line = " ".join(line)
        word_list.append(line)
    return word_list

In [191]:
def hop(datas):
    kkma = Kkma()
    twit = Twitter()
    word_list = []
    for data in datas:
        line = kkma.nouns(data[1])
        line2 = twit.nouns(data[1])
        line = line + line2
        line = " ".join(line)
        word_list.append(line)
    return word_list

In [192]:
extract_noun(X_train[:10])

['도그 도그차일드 차일드 레오파드 레오파드애견신발 애견 신발 노랑 강아지 강아지슈즈 슈즈 애',
 '불스 방청 방청윤활제 윤활제 윤활 윤활방청제 방청제 방청유 유 윤활유 녹',
 '쿠스쿠 쿠스쿠스파이 스파이 블루 베리 치즈 치즈파이 파이 100 8조각 조각',
 '파 파사바체 사바 체 유리 계량컵 주방 주방용품 용품 계량 컵 조리 조리도구 도구 계량용품',
 '제주 코 코코몽에코 몽 에코 파크 이즈 이즈랜드 랜드 제주 제주도 도 관광지 2곳 곳 패키지 입장권 할인 기 기프트제주 프트 승마 승마체험 체험 제주승마할인 승마장 장 추천 가격 가격말 말 말타기',
 '발목 서포트 압박 압박밴드 밴드 발목보호대 보호대 무',
 '중고 서버 0005',
 '투 투핫 핫 로 로키티 키 티 킹 킹컵 컵 세트',
 '인테리어 스위치 스위치커버 커버 그래픽 그래픽스티커 스티커 파인애플',
 '멋 캐주얼 정장 스펜더 멜빵']

In [193]:
hop(X_train[:10])

['도그 도그차일드 차일드 레오파드 레오파드애견신발 애견 신발 노랑 강아지 강아지슈즈 슈즈 애 도 차 일드 레오파드 애견 신발 노랑 강아지 슈즈 애',
 '불스 방청 방청윤활제 윤활제 360 윤활 윤활방청제 방청제 방청유 유 윤활유 녹 불스 방청 윤활제 윤활 방청제 방청 유 윤활유 녹',
 '쿠스쿠 쿠스쿠스파이 스파이 블루 베리 치즈 치즈파이 파이 800 100 8 8조각 조각 쿠스쿠스 파이 블루베리 치즈 파이 조각',
 '파 파사바체 사바 체 유리 계량컵 주방 주방용품 용품 계량 컵 조리 조리도구 도구 계량용품 파사 바체 유리 계량컵 주방용품 계량컵 계량 용컵 조리 도구 계량 용품',
 '제주 코 코코몽에코 몽 에코 파크 이즈 이즈랜드 랜드 제주 제주도 도 관광지 2 2곳 곳 패키지 입장권 할인 기 기프트제주 프트 승마 승마체험 체험 제주승마할인 승마장 장 추천 가격 가격말 말 말타기 제주 코코몽 에코 파크 메이즈랜드 제주도 관광지 곳 패키지 입장권 할인 기프트 제주 제주 승마 체험 제주 승마 제주 승마 할인 제주 승마 추천 제주도 승마 체험 가격 말타 곳 말타 기',
 '537 발목 서포트 압박 압박밴드 밴드 발목보호대 보호대 무 발목 포트 압박 밴드 발목 보호대 무',
 '중고 서버 300 0005 15 중고 서버 용',
 '투 투핫 핫 로 로키티 키 티 킹 킹컵 컵 세트 투핫 헬로키티 헬로키티 쿠킹컵 세트',
 '인테리어 스위치 스위치커버 커버 그래픽 그래픽스티커 스티커 파인애플 7170 인테리어 스위치 커버 그래픽 스티커 파인애플',
 '멋 캐주얼 정장 스펜더 멜빵 1005 멋 캐 주얼 정장 스 펜더 멜빵']

In [194]:
twitter(X_train[:10])

['도 차 일드 레오파드 애견 신발 노랑 강아지 슈즈 애',
 '불스 방청 윤활제 윤활 방청제 방청 유 윤활유 녹',
 '쿠스쿠스 파이 블루베리 치즈 파이 조각',
 '파사 바체 유리 계량컵 주방용품 계량컵 계량 용컵 조리 도구 계량 용품',
 '제주 코코몽 에코 파크 메이즈랜드 제주도 관광지 곳 패키지 입장권 할인 기프트 제주 제주 승마 체험 제주 승마 제주 승마 할인 제주 승마 추천 제주도 승마 체험 가격 말타 곳 말타 기',
 '발목 포트 압박 밴드 발목 보호대 무',
 '중고 서버 용',
 '투핫 헬로키티 헬로키티 쿠킹컵 세트',
 '인테리어 스위치 커버 그래픽 스티커 파인애플',
 '멋 캐 주얼 정장 스 펜더 멜빵']

In [195]:
datas = twitter(X_train[:10])

In [196]:
from googletrans import Translator
translator = Translator()
eng_list = []
for x in datas:
    eng_list.append(translator.translate(x).text)

In [197]:
translator = Translator()
for i, data in enumerate(x_text_list) :
    eng_list.append(translator.translate(data[1]).text)
    if i % 500 == 0:
        print(i)
        with open("train_eng{}.pickle".format(i), 'wb') as f:
            pickle.dump(eng_list, f)
            eng_list = []


0


KeyboardInterrupt: 

In [198]:
def withInSpecials(strings):
    for ch in strings:
        if (ch in string.punctuation):
            return True
    return False

def hop_some(datas):
    kkma = Kkma()
    twit = Twitter()
    word_list = []
    for data in datas:
        line = kkma.nouns(data[1])
        
        for x in line:
            if x in  string.punctuation:
                line.remove()
        line = " ".join(line)
        word_list.append(line)
    return word_list

In [199]:
with open("eng.pickle{}", 'rb') as f:
    pickle.dump(eng_list, f)

FileNotFoundError: [Errno 2] 그런 파일이나 디렉터리가 없습니다: 'eng.pickle{}'

In [ ]:
final = extract_noun

### 파일에서 학습 데이터를 읽는다.

In [20]:
import json


In [200]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [201]:
# joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [202]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [203]:
print(y_name_id_dict)

{'자동차/공구': 1, '디지털': 10, '취미': 5, '뷰티': 0, '도서/문구': 8, '잡화': 14, '출산/육아': 16, '건강': 7, '식품': 6, '반려동물': 4, '가구/인테리어': 11, '여행/e쿠폰': 15, '스포츠/레저': 9, '생필품/주방': 13, '컴퓨터': 3, '가전': 12, '의류': 2}


In [25]:

# y_name_set = set(y_text_list)
# y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
# y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_id_list = [y_name_id_dict[x] for x in y_text_list]


### text 형태로 되어 있는 상품명을 각 단어 id 형태로 변환한다.

In [204]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(final(x_text_list))
y_list = [y_name_id_dict[x] for x in y_text_list]


### train test 분리하는 방법 

In [27]:
print(x_text_list[2])

('1546650266', '(AGCRIP 웨빙 포켓조끼 빅사이즈 조끼 MK321_324 아웃')


In [205]:
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)


In [ ]:
# print(vectorizer.transform(list(map(lambda i : i[1],X_train))))

### 몇개의 파라미터로 간단히 테스트 하는 방법

In [ ]:

# for c in [1,5,10]:
#     clf = LinearSVC(C=c)
#     X_train_text = map(lambda i : i[1],X_train
#     clf.fit(vectorizer.transform(X_train_text), y_train)
#     print (c,clf.score(vectorizer.transform(map(lambda i : i[1],X_test)), y_test))


In [ ]:
final = extract_noun

In [ ]:
X_train_noun = final(X_train)
X_test_noun = final(X_test)

In [ ]:

# for i in range(1, 25):
#     c = 0.07 + (i / 1000)
#     clf = LinearSVC(C=c)
#     X_train_text = map(lambda i : i[1],X_train)
#     clf.fit(vectorizer.transform(X_train_noun), y_train)
#     print (c,clf.score(vectorizer.transform(X_test_noun), y_test))


In [ ]:
# c = 0.081
# clf = LinearSVC(C=c)
# X_train_text = map(lambda i : i[1],X_train)
# clf.fit(vectorizer.transform(X_train_noun), y_train)
# print (c,clf.score(vectorizer.transform(X_test_noun), y_test))

### 최적의 파라미터를 알아서 다 해보고, n-fold cross validation까지 해주는 방법 - GridSearchCV

In [ ]:
x_text_nouns = extract_noun(x_text_list)

In [ ]:
svc_param = np.logspace(-1,1,4)

In [ ]:
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)

In [ ]:
gsvc.fit(vectorizer.transform(x_text_nouns), y_list)

In [ ]:
print(gsvc.best_score_, gsvc.best_params_)

#### 평가 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송

In [ ]:
test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        test_x_text_list.append((info['pid'],info['name']))


In [ ]:
# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],test_x_text_list)))

In [ ]:
pred_list = gsvc.predict(vectorizer.transform(extract_noun(test_x_text_list)))

In [ ]:
import pickle
with open("prediction.pickle", 'rb') as f:
    pred_list = pickle.load(f)

In [ ]:
len(pred_list)

In [ ]:
# print (pred_list.tolist())

In [ ]:
import requests
name='정윤수'
nickname='bro_text_cnn'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


#### eval 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송
 * 실제 여기서 나온 점수로 채점을 한다.

In [ ]:
eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))`

In [ ]:
pred_list = gsvc.predict(vectorizer.transform(extract_noun(eval_x_text_list)))

In [ ]:
# import pickle
# with open("eval_prediction.pickle", 'rb') as f:
#     pred_list = pickle.load(f)
# #     pred_list = ",".join(map(lambda i : str(int(i)),pred_list.tolist()))

In [ ]:
# pred_list

In [ ]:
name='정윤수'
nickname='text_cnn인데 더 안좋네'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


### CNN 으로 추출한 이미지 데이터 사용하기 
 * keras mobilenet 으로 추출한 데이터, 이 데이터를 아래처럼 읽어서 사용 가능함
 * 더 성능이 높은 모델로 이미지 피쳐를 추출하면 성능 향상 가능함 

In [ ]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
        if idx%100 == 0:
            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#         print (line)
#         break
        

In [ ]:
from scipy import sparse 

In [ ]:
img_feature_list = []
for pid, name in X_train:
#     print(pid, name)
    if pid in pid_img_feature_dict:
        img_feature_list.append(pid_img_feature_dict[pid])
#         print (len(pid_img_feature_dict[pid]),type(pid_img_feature_dict[pid]))
#         break
    else:
        img_feature_list.append(np.zeros(1000))
#     break

In [ ]:
img_feature_t_list = []
for pid, name in X_test:
    if pid in pid_img_feature_dict:
        img_feature_t_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_t_list.append(np.zeros(1000))


In [ ]:
img_feature_t_list[1]

#### 아래 부분은 text feature 와 이미지 feature 를 합쳐서 feature 를 만드는 부분이다. 이 부분에 대해서는 각자 한번 합치는 방법을 찾아 보면 된다. 

In [ ]:
concat_x_list = sparse.hstack((vectorizer.transform(X_train_noun), img_feature_list),format='csr')
concat_t_x_list = sparse.hstack((vectorizer.transform(X_test_noun), img_feature_t_list), format='csr')


In [ ]:
concat_refined_list = sparse.hstack((vectorizer.transform(X_train_noun + X_test_noun), img_feature_list + img_feature_t_list), format='csr')

In [ ]:
for i in range(1,25):
    c = 0.01 + (i / 300)
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_x_list, y_train)
    print (c,clf2.score(concat_t_x_list, y_test))


In [ ]:
c = 0.021
clf2 = LinearSVC(C=c)
clf2.fit(concat_refined_list, y_train + y_test)
print (c,clf2.score(concat_t_x_list, y_test))

In [ ]:
svc_param = np.logspace(-1 , 1, 6)


In [ ]:
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)


In [ ]:
gsvc.fit(concat_refined_list, y_train + y_test)

In [ ]:
print(gsvc.best_score_, gsvc.best_params_)

In [ ]:
del pid_img_feature_dict


### CNN 피쳐를 추가 해서 분류후 평가 서버에 분류 결과를 전송 

In [ ]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
        if idx%100 == 0:
            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#         print (line)
#         break
        

In [ ]:
test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        test_x_text_list.append((info['pid'],info['name']))

# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [ ]:
img_feature_test_list = []

In [ ]:
for pid, name in test_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_test_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_test_list.append(np.zeros(1000))


In [ ]:
x_test_feature_list = vectorizer.transform(extract_noun(test_x_text_list))

#### 2개 feature 를 합치는 방법 찾아보기 

In [ ]:
concat_test_x_list = sparse.hstack((x_test_feature_list, img_feature_test_list),format='csr')

In [ ]:
# pred_list = clf2.predict(concat_test_x_list)

In [ ]:
pred_list = gsvc.predict(concat_test_x_list)

In [ ]:

import requests
name='정윤수'
nickname='brother'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



In [ ]:
eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))

# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [ ]:
img_feature_eval_list = []
for pid, name in eval_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))


In [ ]:
x_feature_list = vectorizer.transform(final(eval_x_text_list))

In [ ]:
concat_eval_x_list = sparse.hstack((x_feature_list, img_feature_eval_list),format='csr')

In [ ]:
pred_list = clf2.predict(concat_eval_x_list)

In [ ]:
pred_list = gsvc.predict(concat_eval_x_list)

In [ ]:

import requests
name='정윤수'
nickname='brother_svm_'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



# 변수모음

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=60)
X_train_noun = extract_noun(X_train)
X_test_noun = extract_noun(X_test)

mode_test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        mode_test_x_text_list.append((info['pid'],info['name']))
        
mode_eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        mode_eval_x_text_list.append((info['pid'],info['name']))

mode_test_x_noun = extract_noun(mode_test_x_text_list)
mode_eval_x_noun = extract_noun(mode_eval_x_text_list)
        
pid_img_feature_train_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
        if idx%100 == 0:
            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_train_dict[pid] = img_feature

img_feature_train_list = []
for pid, name in X_train:
    if pid in pid_img_feature_train_dict:
        img_feature_train_list.append(pid_img_feature_train_dict[pid])
    else:
        img_feature_train_list.append(np.zeros(1000))

img_feature_test_list = []
for pid, name in X_test:
    if pid in pid_img_feature_dict:
        img_feature_test_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_test_list.append(np.zeros(1000))

        
pid_img_feature_eval_dict = {}
with open("refined_category_dataset.img_feature.eval.dat") as fin:
    for idx,line in enumerate(fin):
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_eval_dict[pid] = img_feature
#         print (line)
#         break


# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
img_feature_mode_test_list = []
for pid, name in mode_test_x_text_list:
    if pid in pid_img_feature_eval_dict:
        img_feature_mode_test_list.append(pid_img_feature_eval_dict[pid])
    else:
        img_feature_mode_test_list.append(np.zeros(1000))
        
        
# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
img_feature_eval_list = []
for pid, name in mode_eval_x_text_list:
    if pid in pid_img_feature_eval_dict:
        img_feature_eval_list.append(pid_img_feature_eval_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))
        


concat_x_train_list = sparse.hstack((vectorizer.transform(X_train_noun), img_feature_train_list))
concat_x_test_list = sparse.hstack((vectorizer.transform(X_test_noun), img_feature_test_list))

x_feature_eval_list = vectorizer.transform(mode_eval_x_noun)
x_feature_test_list = vectorizer.transform(mode_test_x_noun)
concat_x_eval_list = sparse.hstack((x_feature_eval_list, img_feature_eval_list),format='csr')
concat_x_mode_test_list = sparse.hstack((x_feature_test_list, img_feature_mode_test_list),format='csr')

In [ ]:
concat_refi_list = sparse.hstack((vectorizer.transform(X_train_noun + X_test_noun), img_feature_train_list + img_feature_test_list))

In [ ]:
for i in range(1, 25):
    c = 0.072 + (i / 1000)
    clf = LinearSVC(C=c)
    clf.fit(vectorizer.transform(X_train_noun), y_train)
    print (c,clf.score(vectorizer.transform(X_test_noun), y_test))
    
    


In [ ]:
c = 0.094
clf = LinearSVC(C=c)
clf.fit(vectorizer.transform(X_train_noun), y_train)
print (c,clf.score(vectorizer.transform(X_test_noun), y_test))
pred_list = clf.predict(vectorizer.transform(mode_test_x_noun))

In [ ]:
c = 0.094
clf2 = LinearSVC(C=c)
clf2.fit(concat_x_train_list, y_train)
print (c,clf2.score(concat_x_test_list, y_test))

In [ ]:
pred_list = clf2.predict(concat_x_mode_test_list)

In [ ]:
pred_list = gsvc.predict(concat_x_mode_test_list)

In [ ]:
import requests
name='정윤수'
nickname='편brew5_svm'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


In [ ]:
for i in range(1,25):
    c = 0.10 + (i / 1000)
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_x_train_list, y_train)
    print (c,clf2.score(concat_x_test_list, y_test))


In [ ]:
for i in range(1,25):
    c = 0.10 + (i / 100)
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_refi_list, y_train + y_test)
    print (c,clf2.score(concat_x_test_list, y_test))


In [ ]:
c = 0.30
clf2 = LinearSVC(C=c)
clf2.fit(concat_refi_list, y_train + y_test)
print (c,clf2.score(concat_x_test_list, y_test))

In [ ]:
pred_list = clf2.predict(concat_x_eval_list)

In [ ]:

import requests
name='정윤수'
nickname='편brew_svm'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())

